# Projeto final

Este projeto tem como objetivo desenvolver uma solução para o desafio [RecSys Challenge 2018](http://www.recsyschallenge.com/2018/)

Disciplina: EEL 410250 - Aprendizado de Máquina
Aluno: Gustavo de Paula Santos
Matrícula: 19100833